## Observations and Insights 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
merged_data = pd.merge(mouse_metadata, study_results, how = 'outer', on = 'Mouse ID')
# Display the data table for preview
merged_data.head(20)

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
5,k403,Ramicane,Male,21,16,25,33.464577,1
6,k403,Ramicane,Male,21,16,30,31.099498,1
7,k403,Ramicane,Male,21,16,35,26.546993,1
8,k403,Ramicane,Male,21,16,40,24.365505,1
9,k403,Ramicane,Male,21,16,45,22.050126,1


In [2]:
# Checking the number of mice.
mice_total = len(merged_data.groupby('Mouse ID').count())
mice_total

249

In [3]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
merged_data['Mouse ID'].value_counts()
#g989 has too many entries!!

g989    13
l897    10
p189    10
u364    10
p136    10
        ..
v199     1
t573     1
d133     1
x336     1
n482     1
Name: Mouse ID, Length: 249, dtype: int64

In [4]:
# Optional: Get all the data for the duplicate mouse ID. 
merged_data.groupby('Mouse ID').get_group('g989').head(14)
#it can be seen that data for this Mouse ID has duplicate timepoints from 0-20 and they are
#reporting dicordant Tumor Volumes and Metastatic Sites for those duplicate Timepoints . 

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
908,g989,Propriva,Female,21,26,0,45.000000,0
909,g989,Propriva,Female,21,26,0,45.000000,0
910,g989,Propriva,Female,21,26,5,48.786801,0
911,g989,Propriva,Female,21,26,5,47.570392,0
912,g989,Propriva,Female,21,26,10,51.745156,0
913,g989,Propriva,Female,21,26,10,49.880528,0
914,g989,Propriva,Female,21,26,15,51.325852,1
915,g989,Propriva,Female,21,26,15,53.442020,0
916,g989,Propriva,Female,21,26,20,55.326122,1
917,g989,Propriva,Female,21,26,20,54.657650,1


In [5]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
cleaned_merged = merged_data.drop([908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920])
cleaned_merged.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [6]:
# Checking the number of mice in the clean DataFrame.
mice_total= len(cleaned_merged['Mouse ID'].value_counts())
mice_total


248

## Summary Statistics

In [14]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
summaries_list = []
# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.
for regimen in cleaned_merged.groupby('Drug Regimen'):
    summary = pd.DataFrame({
        'regimen' : [regimen[0]], 
        'mean' : [regimen[1]['Tumor Volume (mm3)'].mean()], 
        'median' : [regimen[1]['Tumor Volume (mm3)'].median()], 
        'variance' : [regimen[1]['Tumor Volume (mm3)'].var()], 
        'standard deviation' : [regimen[1]['Tumor Volume (mm3)'].std()]
    }
    )
    
    summaries_list.append(summary)
    print(summary)
pd.DataFrame(index = summaries_list)


     regimen       mean     median   variance  standard deviation
0  Capomulin  40.675741  41.557809  24.947764            4.994774
    regimen       mean     median   variance  standard deviation
0  Ceftamin  52.591172  51.776157  39.290177            6.268188
     regimen       mean     median   variance  standard deviation
0  Infubinol  52.884795  51.820584  43.128684            6.567243
    regimen       mean     median   variance  standard deviation
0  Ketapril  55.235638  53.698743  68.553577            8.279709
    regimen       mean     median   variance  standard deviation
0  Naftisol  54.331565  52.509285  66.173479            8.134708
   regimen       mean     median   variance  standard deviation
0  Placebo  54.033581  52.288934  61.168083            7.821003
    regimen      mean     median   variance  standard deviation
0  Propriva  52.32093  50.446266  43.852013            6.622085
    regimen       mean     median   variance  standard deviation
0  Ramicane  40.216745  4

,,,,,,,,,
regimen,regimen,regimen,regimen,regimen,regimen,regimen,regimen,regimen,regimen
mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
median,median,median,median,median,median,median,median,median,median
variance,variance,variance,variance,variance,variance,variance,variance,variance,variance
standard deviation,standard deviation,standard deviation,standard deviation,standard deviation,standard deviation,standard deviation,standard deviation,standard deviation,standard deviation


In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line


## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pandas.



In [ ]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pyplot.



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
